In [1]:
#!/usr/bin/env python
# coding: utf-8

import numpy as np
import sys
import pandas as pd




In [2]:
fileNames = ['predictions_SVR_RBF_TotalCost.csv',\
            'predictions_eRNN_TotalCost.csv',\
            'predictions_MLP_TotalCost.csv',
            'predictions_LSTM_TotalCost.csv',]

In [3]:
inputFolder = '../Prediction_Files/'
# dataPred = pd.read_csv('../Prediction_Files/arimaNormalizedPrediction.csv')
outputDirectory = './results3/'
outputFileName = outputDirectory + "rewardAwareUtility-SVR_Linear.csv"

# reading predictions and transforming data

In [4]:
def rowToTime(row):
    timeAtMoment = row.Hours*3600*1000
    timeAtMoment += row.Minutes*60*1000
    timeAtMoment += row.Seconds*1000
    return timeAtMoment

In [5]:
def shiftTransformCol50(data):
    return (data + abs(data.min()))*50

In [6]:
def utilFunction(latency, cost, reward = 100, reliability = 1):
    epsilon = 0.00000000000000000000000000000000000000000000001

    if reliability == 0:
        return 0
    else:
        return ((reward)/(latency+cost+epsilon)) * reliability 
    


realUtilityMean = 4.138921e+06	
realUtilStd = 6.175693e+06	

In [24]:
def celia(data,updateThreshold = 1000):
    lastUpdate = 0;
    utilitPredicted = []
    utilityReal = []
    award = []
    lostUtility = []
    

    temp = data.iloc[1]['Timestamp'] - data.iloc[0]['Timestamp']

    for i in range(1,len(data)):
#     timeDeltaReward = (data.iloc[i]['Timestamp'] - data.iloc[i-1]['Timestamp']).total_seconds()
        timeDeltaReward = (data.iloc[i]['Timestamp'] - data.iloc[lastUpdate]['Timestamp'])

        predictedUtil = utilFunction(data.iloc[i].predLatency, data.iloc[i].predicted_Total_Cost,(float)(timeDeltaReward), data.iloc[i].predReliability )

        realUtility = utilFunction(data.iloc[i].Latency, data.iloc[i].Total_Cost,(float)(timeDeltaReward), data.iloc[i].Reliability )
        
        utilitPredicted.append(predictedUtil)
        
        utilityReal.append(realUtility)
        
        
        if predictedUtil >= updateThreshold:
    #         award.append(award[i-1] + realUtility-1);
            award.append(realUtility-updateThreshold);
            lastUpdate = i;
            lostUtility.append(0)
        else:
    #         award.append(award[i-1]);
            award.append(0);

            if realUtility >= updateThreshold:
                lostUtility.append(realUtility)
            else:
                lostUtility.append(0) 
    
    results = pd.DataFrame(np.stack([utilitPredicted,utilityReal,award,lostUtility],axis=1),columns=['predictedUtility','realUtility','award','lostUtility'])
    return results
#     results.to_csv("./results/dummyRandom.csv")

In [19]:
def celiaSim(dataPred):
    ### this is converting hours, minutes, seconds to a millisecond time series
    dataPred.Hours = ((dataPred.Hours)*23)+0
    dataPred.Minutes = ((dataPred.Minutes)*59)+0
    dataPred.Seconds = ((dataPred.Seconds)*59.999805)+0.1025061111111
    
    cumulativeTime = [rowToTime(dataPred.iloc[0])]
    resetAddTime = 23*3600*1000
    resetAddTime += 59*60*1000
    resetAddTime += 60*1000
        
    for i in range(1,len(dataPred)):
        timeAtMoment = rowToTime(dataPred.iloc[i])
        lastTimeMoment = rowToTime(dataPred.iloc[i-1])
        if timeAtMoment > lastTimeMoment:
            cumuTime = cumulativeTime[i-1] + (timeAtMoment - lastTimeMoment)
            cumulativeTime.append(cumuTime)
        else:
            cumulativeTime.append(cumulativeTime[i-1]+(timeAtMoment+resetAddTime)-lastTimeMoment)
    #         print(cumulativeTime[i])
    
    cumulativeTime = cumulativeTime- cumulativeTime[0]
    
    ### shifting and transforming data to a range of 0-50
    newData = np.stack((cumulativeTime, shiftTransformCol50(dataPred.Latency), shiftTransformCol50(dataPred.Total_Cost), dataPred.Reliability, shiftTransformCol50(dataPred.predicted_Latency), shiftTransformCol50(dataPred.predicted_Total_Cost), round(dataPred.predicted_Reliability)),axis = 1)
    newData = pd.DataFrame(newData, columns=['Timestamp', 'Latency', 'Cost', 'Reliability', 'predLatency', 'predCost', 'predReliability' ])

    return celia(newData)
    

In [20]:
# result = celiaSim(dataPred)


In [21]:
def predictionStatAnalysis(data):
    cost = abs(data.Cost - data.predicted_Cost).describe()
    latency = abs(data.Latency - data.predicted_Latency).describe()
    reliability = abs(data.Reliability - data.predicted_Reliability).describe()
    for i in range(1,len(cost)):
        print (cost[i])
    print('\n')
    for i in range(1,len(cost)):
        print (latency[i])

In [22]:
# result.to_csv(outputDirectory+'result_'+i)


In [23]:
celiaStats = []
predictionStats = []
for i in fileNames:
    dataPred = pd.read_csv(inputFolder+i)
    result = celiaSim(dataPred)
#     result.to_csv(outputDirectory+'celiaResult_'+i)
    utilDif = np.sum(abs(result.predictedUtility-result.realUtility))
    award = sum(result.award)
    lostUtil = sum(result.lostUtility)
    celiaStats.append([i,utilDif, award, lostUtil])

AttributeError: 'Series' object has no attribute 'predicted_Total_Cost'

In [ ]:
celiaStats = pd.DataFrame(celiaStats,columns=['predictionType','utilDif', 'award','lostUtil'])
# celiaStats.to_csv(outputDirectory+'totalCostStats.csv')

In [ ]:
celiaStats

In [ ]:
cost of doing business

add 1/0 col